In [1]:
#libraries 
import scipy.stats as stats
import numpy as np
import pandas as pd
# from sklearn.model_selection import 
# visual
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import r2_score, accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix, accuracy_score, plot_roc_curve, roc_auc_score, recall_score, precision_score, f1_score

%matplotlib inline

## Import Data

In [2]:
train_zero = pd.read_csv("../assets/clean_zero_train.csv")
train_mean = pd.read_csv("../assets/clean_mean_train.csv")
test = pd.read_csv("../assets/clean_zero_test.csv")

In [3]:
train_zero.set_index('id', inplace = True)
train_zero.head()

,mssubclass,lotfrontage,lotarea,lotshape,landslope,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,...,conlw_saletype,conli_saletype,conld_saletype,oth_saletype,normal_salecondition,abnorml_salecondition,adjland_salecondition,alloca_salecondition,family_salecondition,partial_salecondition
id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,3,2,7,5,2003,2003,196.0,...,0,0,0,0,1,0,0,0,0,0
2,20,80.0,9600,3,2,6,8,1976,1976,0.0,...,0,0,0,0,1,0,0,0,0,0
3,60,68.0,11250,2,2,7,5,2001,2002,162.0,...,0,0,0,0,1,0,0,0,0,0
4,70,60.0,9550,2,2,7,5,1915,1970,0.0,...,0,0,0,0,0,1,0,0,0,0
5,60,84.0,14260,2,2,8,5,2000,2000,350.0,...,0,0,0,0,1,0,0,0,0,0


In [4]:
train_mean.set_index('id', inplace = True)
train_mean.head()

,mssubclass,lotfrontage,lotarea,lotshape,landslope,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,...,conlw_saletype,conli_saletype,conld_saletype,oth_saletype,normal_salecondition,abnorml_salecondition,adjland_salecondition,alloca_salecondition,family_salecondition,partial_salecondition
id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,3,2,7,5,2003,2003,196.0,...,0,0,0,0,1,0,0,0,0,0
2,20,80.0,9600,3,2,6,8,1976,1976,0.0,...,0,0,0,0,1,0,0,0,0,0
3,60,68.0,11250,2,2,7,5,2001,2002,162.0,...,0,0,0,0,1,0,0,0,0,0
4,70,60.0,9550,2,2,7,5,1915,1970,0.0,...,0,0,0,0,0,1,0,0,0,0
5,60,84.0,14260,2,2,8,5,2000,2000,350.0,...,0,0,0,0,1,0,0,0,0,0


In [5]:
test.set_index('id', inplace = True)
test.head()

,mssubclass,lotfrontage,lotarea,lotshape,landslope,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,...,conlw_saletype,conli_saletype,conld_saletype,oth_saletype,normal_salecondition,abnorml_salecondition,adjland_salecondition,alloca_salecondition,family_salecondition,partial_salecondition
id,,,,,,,,,,,,,,,,,,,,,
1461,20,80.0,11622,3,2,5,6,1961,1961,0.0,...,0,0,0,0,1,0,0,0,0,0
1462,20,81.0,14267,2,2,6,6,1958,1958,108.0,...,0,0,0,0,1,0,0,0,0,0
1463,60,74.0,13830,2,2,5,5,1997,1998,0.0,...,0,0,0,0,1,0,0,0,0,0
1464,60,78.0,9978,2,2,6,6,1998,1998,20.0,...,0,0,0,0,1,0,0,0,0,0
1465,120,43.0,5005,2,2,8,5,1992,1992,0.0,...,0,0,0,0,1,0,0,0,0,0


# Selecting our Features for Zeroed Data

In [6]:
# features = ['overall_qual','exter_qual', 'gr_liv_area', 'garage_overall']
features = [col for col in train_zero._get_numeric_data().columns if col !='saleprice']
# This selects features in the columns that are NOT saleprice and likely inaccurate

X = train_zero[features]
y = train_zero['saleprice']

## Train/test split

Train test and split the data to split our data into a training set and a holdout set for us to verify the model's accuracy.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)

## Scaling  
---  
We may potentially use Lasso and Ridge regression models, thus we will scale them to run our tests.

In [8]:
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

## Create a function to get our Model Evaluation Metrics

In [9]:
def model_metrics(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)
    
    # Cross Validation Score
    print("Cross Validation Score: " + str(cross_val_score(model, X_train, y_train, cv=5).mean()))
    
    # R2 Scores
    print("Train R2 Score: " + str(metrics.r2_score(y_train, model.predict(X_train)))) # Train Score
    print("Test R2 Score: " + str(metrics.r2_score(y_test, model.predict(X_test)))) # Test Score
    

    
    return None

## Instantiate our models
--- 
Let's instantiate our regression models.

In [10]:
lr = LinearRegression()
model_metrics(X_train, y_train,X_test, y_test, lr)

Cross Validation Score: -2.201537000900645e+22
Train R2 Score: 0.930398849074895
Test R2 Score: -5.281709389248083e+21


In [11]:
lasso = LassoCV(n_alphas = 400)
model_metrics(X_train, y_train,X_test, y_test, lasso)

Cross Validation Score: 0.7885286665048632
Train R2 Score: 0.9083719551968772
Test R2 Score: 0.8389498191533531


In [12]:
ridge = RidgeCV(alphas=np.linspace(.1,10,150))
model_metrics(X_train, y_train,X_test, y_test, ridge)

Cross Validation Score: 0.770686024986392
Train R2 Score: 0.9301379365292022
Test R2 Score: 0.8218312319943738


In [13]:
LogReg = LogisticRegression(random_state = 0)
model_metrics(X_train, y_train,X_test, y_test, LogReg)

C:\Users\ACCEP7ANCE\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\ACCEP7ANCE\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\ACCEP7ANCE\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or

Cross Validation Score: 0.0182648401826484
Train R2 Score: 0.9997019602334359
Test R2 Score: 0.5809948520891234


C:\Users\ACCEP7ANCE\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Selecting our Features for Imputed Mean Data

In [14]:
# features = ['overall_qual','exter_qual', 'gr_liv_area', 'garage_overall']
features = [col for col in train_zero._get_numeric_data().columns if col !='saleprice']
# This selects features in the columns that are NOT saleprice and likely inaccurate

X = train_mean[features]
y = train_mean['saleprice']

## Train/test split

Train test and split the data to split our data into a training set and a holdout set for us to verify the model's accuracy.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)

## Scaling  
---  
We may potentially use Lasso and Ridge regression models, thus we will scale them to run our tests.

In [16]:
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

## Instantiate our models
--- 
Let's instantiate our regression models.

In [17]:
lr = LinearRegression()
model_metrics(X_train, y_train,X_test, y_test, lr)

Cross Validation Score: -2.201537000900645e+22
Train R2 Score: 0.930398849074895
Test R2 Score: -5.281709389248083e+21


In [18]:
lasso = LassoCV(n_alphas = 400)
model_metrics(X_train, y_train,X_test, y_test, lasso)

Cross Validation Score: 0.7885286665048632
Train R2 Score: 0.9083719551968772
Test R2 Score: 0.8389498191533531


In [19]:
ridge = RidgeCV(alphas=np.linspace(.1,10,150))
model_metrics(X_train, y_train,X_test, y_test, ridge)

Cross Validation Score: 0.770686024986392
Train R2 Score: 0.9301379365292022
Test R2 Score: 0.8218312319943738


In [20]:
LogReg = LogisticRegression(random_state = 0)
model_metrics(X_train, y_train,X_test, y_test, LogReg)

C:\Users\ACCEP7ANCE\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\ACCEP7ANCE\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\ACCEP7ANCE\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or

Cross Validation Score: 0.0182648401826484
Train R2 Score: 0.9997019602334359
Test R2 Score: 0.5809948520891234


C:\Users\ACCEP7ANCE\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Appears that LassoCV is the best predictor for our models, and that our A/B testing yielded no results as it seems to have no effect on our Cross Validation score or our R2 scores. Let's use our LassoCV to make a prediction with our cleaned Test Data

In [21]:
lasso.fit(X_train, y_train)
test_pred = lasso.predict(test)
test_id = np.array(test.index).astype(int)

In [27]:
lasso.fit(X_train, y_train)
train_pred = lasso.predict(X_train)
train_pred

array([316701.06834188, 139800.78180644, 151672.11514084, ...,
       305778.82903784, 110503.15260875, 273352.58523556])

In [22]:
test_pred_df = pd.DataFrame(test_pred, test_id, columns = ["SalePrice"])

# Set 'id' as the index
test_pred_df.index.name = 'Id'

# Viewing the first 5 prediction probabilities
test_pred_df.head()

,SalePrice
Id,
1461,9.620155e+07
1462,1.255100e+08
1463,1.301199e+08
1464,1.136221e+08
1465,8.406577e+07


In [23]:
test_pred_df.to_csv('../assets/test_pred_lasso.csv', index=True)

---